<a href="https://colab.research.google.com/github/bashirnubtk/Virtual-CPU-Emulator/blob/main/Final%20submission/Final_Virtual_Cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import logging

class VirtualCPU:
    def __init__(self):
        # Registers (Extended for Week 10 Features)
        self.registers = [0] * 16  # 16 General-purpose registers
        self.pc = 0  # Program counter
        self.memory = [0] * 256  # 256-byte memory
        self.running = True  # Flag to keep CPU running
        self.stack = []  # Stack for subroutine calls
        self.breakpoints = set()  # Breakpoints for debugging
        self.step_mode = False  # Step execution mode

        # Logging setup
        logging.basicConfig(filename="cpu_log.txt", level=logging.INFO,
                            format='%(asctime)s - %(levelname)s - %(message)s')

    def load_program(self, program):
        """ Load program into memory """
        for i, instruction in enumerate(program):
            if i < len(self.memory):
                self.memory[i] = instruction
            else:
                print(f"Memory overflow: Cannot load instruction at position {i}")

    def fetch(self):
        """ Fetch instruction from memory """
        if self.pc < len(self.memory):
            instruction = self.memory[self.pc]
            self.pc += 1
            return instruction
        else:
            print("Error: Program counter out of bounds.")
            self.running = False
            return None

    def binary_repr(self, value):
        """ Return 8-bit binary representation of a number """
        return format(value, '08b')

    def execute(self, instruction):
        """ Execute the fetched instruction """
        if instruction is None:
            return

        opcode = instruction[0]
        if opcode == "LOAD":
            reg, addr = instruction[1], instruction[2]
            self.registers[reg] = self.memory[addr]
            print(f"LOAD: R{reg} = {self.memory[addr]} (Binary: {self.binary_repr(self.memory[addr])})")

        elif opcode == "STORE":
            addr, reg = instruction[1], instruction[2]
            self.memory[addr] = self.registers[reg]
            print(f"STORE: Memory[{addr}] = {self.registers[reg]} (Binary: {self.binary_repr(self.registers[reg])})")

        elif opcode == "ADD":
            reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]
            self.registers[reg1] = self.registers[reg2] + self.registers[reg3]
            print(f"ADD: R{reg1} = R{reg2} ({self.binary_repr(self.registers[reg2])}) + R{reg3} ({self.binary_repr(self.registers[reg3])}) → {self.registers[reg1]} (Binary: {self.binary_repr(self.registers[reg1])})")

        elif opcode == "SUB":
            reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]
            self.registers[reg1] = self.registers[reg2] - self.registers[reg3]
            print(f"SUB: R{reg1} = R{reg2} ({self.binary_repr(self.registers[reg2])}) - R{reg3} ({self.binary_repr(self.registers[reg3])}) → {self.registers[reg1]} (Binary: {self.binary_repr(self.registers[reg1])})")

        elif opcode == "MULT":
            reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]
            self.registers[reg1] = self.registers[reg2] * self.registers[reg3]
            print(f"MULT: R{reg1} = R{reg2} ({self.binary_repr(self.registers[reg2])}) * R{reg3} ({self.binary_repr(self.registers[reg3])}) → {self.registers[reg1]} (Binary: {self.binary_repr(self.registers[reg1])})")

        elif opcode == "JUMP":
            addr = instruction[1]
            self.pc = addr
            print(f"JUMP to {addr}")

        elif opcode == "BRANCH_IF_ZERO":
            reg, addr = instruction[1], instruction[2]
            if self.registers[reg] == 0:
                self.pc = addr
                print(f"BRANCH_IF_ZERO: Jumping to {addr} because R{reg} is 0")

        elif opcode == "CALL":
            addr = instruction[1]
            self.stack.append(self.pc)
            self.pc = addr
            print(f"CALL: Subroutine at {addr}")

        elif opcode == "RETURN":
            if self.stack:
                self.pc = self.stack.pop()
                print(f"RETURN to {self.pc}")

        elif opcode == "HALT":
            self.running = False
            print("HALT: Stopping execution")

        elif opcode == "INPUT":
            reg = instruction[1]
            self.registers[reg] = int(input("Enter value: "))
            print(f"INPUT: R{reg} = {self.registers[reg]} (Binary: {self.binary_repr(self.registers[reg])})")

        elif opcode == "OUTPUT":
            reg = instruction[1]
            print(f"OUTPUT: R{reg} = {self.registers[reg]} (Binary: {self.binary_repr(self.registers[reg])})")

        else:
            print(f"Unknown opcode: {opcode}")

        # Log execution step
        logging.info(f"Executed: {instruction}")

    def set_breakpoint(self, addr):
        """ Set a breakpoint at a specific address """
        self.breakpoints.add(addr)

    def enable_step_mode(self):
        """ Enable step execution mode """
        self.step_mode = True

    def run(self):
        """ Run the CPU """
        while self.running:
            if self.pc in self.breakpoints:
                print(f"Breakpoint hit at {self.pc}. Press Enter to continue...")
                input()

            instruction = self.fetch()
            if isinstance(instruction, tuple):
                self.execute(instruction)
            else:
                print(f"Error: {instruction} is not a valid instruction")
                break

            if self.step_mode:
                input("Press Enter for next step...")

# Example program with input/output and multiplication
program = [
    ("INPUT", 0),         # Take input into register 0
    ("INPUT", 1),         # Take input into register 1
    ("MULT", 2, 0, 1),    # Multiply register 0 and 1, store in register 2
    ("OUTPUT", 2),        # Print result from register 2
    ("HALT",)
]

# Create and run the CPU
cpu = VirtualCPU()
cpu.load_program(program)
cpu.run()

# Display the state of the registers and memory
print("Registers:", cpu.registers)


Enter value: 2
INPUT: R0 = 2 (Binary: 00000010)
Enter value: 2
INPUT: R1 = 2 (Binary: 00000010)
MULT: R2 = R0 (00000010) * R1 (00000010) → 4 (Binary: 00000100)
OUTPUT: R2 = 4 (Binary: 00000100)
HALT: Stopping execution
Registers: [2, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
